# NLP. Tarea 5: Modelo del Lenguaje Neuronal.

**Diego Moreno**

In [27]:
# Tools
import os
import time
import shutil
import random
from typing import Tuple
from argparse import Namespace
import matplotlib.pyplot as plt
from itertools import permutations
from random import shuffle
# Preprocesing
import nltk
from nltk.corpus import stopwords
from nltk import ngrams
from nltk.tokenize import  TweetTokenizer
from nltk import FreqDist
import pandas as pd
import numpy as np
# Pytorch
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.nn.functional as F
# Scikitlearn
from sklearn.metrics import accuracy_score

In [2]:
seed = 1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False 

In [4]:
pth = ''
X_train = pd.read_csv(pth+'mex_train.txt', sep='\r\n',  engine='python', header=None).loc[:,0].values.tolist()
X_val = pd.read_csv(pth+'mex_val.txt', sep='\r\n',  engine='python', header=None).loc[:,0].values.tolist()

In [34]:
X_val

['así debería ser siempre para que se mueran a la verga',
 'cada día me siento como un perro ovejero tratando de cuidar sus ovejas vale madre pinches rateros póngase a trabajar.😤',
 'hijo de tu puta madre nadamas te la pasas mamando pinche wilo de closet',
 'soñé horrible espero no se cumpla putos temblores.',
 '@usuario mejor vas y la chingas tú veo tus publicaciones y solo denotan inconformidad como típico ciudadano quejumbroso que no hace nada por cambiar',
 'y quizás este loca y quizás me guste estarlo... #siempreneruda',
 'me tienen hasta la verga con sus putos #boomerang pendejos y sin sentido.',
 'cuando va a ser el día que encuentre a alguien en quien pueda  confiar posts la re puta madre?',
 '#ahoritaestoypensando  como putas puede pasar esto?',
 '#todoibabienpero valio madre ya me atacó el insomnio🙅',
 '🤗es mejor así 🤗👌🏻 a chingar a su madre toda persona que solo de migajas 🤗',
 '@usuario que ya chinguen a su madre todos alv',
 'podríamos estar caminando de la mano en paris p

In [68]:
args = Namespace()
args.N = 6

In [95]:
class NgramData():
    def __init__(self, N: int, vocab_max: int=5000, tokenizer=None, embedding_model=None, char_level=False):
        self.tokenizer = tokenizer if tokenizer else self.default_tokenizer
        self.punct = set(['.',',',';',':','-','^','«','»','"','!','¡','?','¿','\'','...','<url>','*','@usuario'])
        self.N = N
        self.vocab_max = vocab_max
        self.UNK = '<unk>'
        self.SOS = '<s>'
        self.EOS = '</s>'
        self.embedding_model = embedding_model
        self.char_level = char_level

    def default_tokenizer(self, doc: str) -> list:
        return doc.split(' ')

    def get_vocab_size(self) -> int:
        return len(self.vocab)

    def remove_word(self, word: str) -> bool:
        word = word.lower()
        is_punct = True if word in self.punct else False
        is_digit = word.isnumeric()
        return is_punct or is_digit

    def get_vocab(self, corpus: list) -> set:
        freq_dist = FreqDist([w.lower() for sent in corpus \
                              for w in self.tokenizer(sent) \
                              if not self.remove_word(w)])
        sorted_words = self.sortFreqDict(freq_dist)[:self.vocab_max-3]
        return set(sorted_words)
    
    def get_vocab_char(self, corpus: list) -> set:
        freq_dist = FreqDist([c for sent in corpus \
                              for c in sent.lower() \
                              if not self.remove_word(c)])
        sorted_words = self.sortFreqDict(freq_dist)[:self.vocab_max-3]
        return set(sorted_words)

    def sortFreqDict(self, freq_dist) -> list:
        freq_dist = dict(freq_dist)
        return sorted(freq_dist, key=freq_dist.get, reverse=True)

    def fit(self, corpus: list) -> None:
        if self.char_level:
            self.vocab = self.get_vocab_char(corpus)
        else:
            self.vocab = self.get_vocab(corpus)
        self.vocab.add(self.UNK)
        self.vocab.add(self.SOS)
        self.vocab.add(self.EOS)
        #print(self.vocab)

        self.w2id = {}
        self.id2w = {}

        if self.embedding_model is not None:
            self.embedding_matrix = np.empty([len(self.vocab), self.embedding_model.vector_size])

        ID = 0
        for doc in corpus:
            if self.char_level:
                for c in doc:
                    if c in self.vocab and not c in self.w2id:
                        self.w2id[c] = ID
                        self.id2w[ID] = c
                        if self.embedding_model is not None:
                            if c in self.embedding_model:
                                self.embedding_matrix[ID] = self.embedding_model[c]
                            else:
                                self.embedding_matrix[ID] = np.random.rand(self.embedding_model.vector_size) 
                        ID += 1
            else:
                for word in self.tokenizer(doc):
                    word_ = word.lower()
                    if word_ in self.vocab and not word_ in self.w2id:
                        self.w2id[word_] = ID
                        self.id2w[ID] = word_
                        if self.embedding_model is not None:
                            if word_ in self.embedding_model:
                                self.embedding_matrix[ID] = self.embedding_model[word_]
                            else:
                                self.embedding_matrix[ID] = np.random.rand(self.embedding_model.vector_size) 
                        ID += 1
        #Special tokens  
        self.w2id.update({self.UNK: ID, 
                          self.SOS: ID+1,
                          self.EOS: ID+2})  
        self.id2w.update({ID  : self.UNK, 
                          ID+1: self.SOS,
                          ID+2: self.EOS})
    
    def replace_unk(self, doc_tokens: list) -> list: 
        if self.char_level:
            for i, token in enumerate(doc_tokens):
                for c in token.lower():
                    if c not in self.vocab:
                        doc_tokens[i] = self.UNK
        else:
            for i, token in enumerate(doc_tokens):
                if token.lower() not in self.vocab:
                    doc_tokens[i] = self.UNK
        return doc_tokens


    def get_ngram_doc(self, doc:str) -> list:
        if self.char_level:
            doc_tokens = 
        doc_tokens = self.tokenizer(doc)
        doc_tokens = self.replace_unk(doc_tokens)
        if self.char_level:
            doc_tokens = [c for w in doc_tokens for c in w.lower()]
        else:
            doc_tokens = [w.lower() for w in doc_tokens]
        doc_tokens = [self.SOS]*(self.N - 1) + doc_tokens + [self.EOS]
        return list(ngrams(doc_tokens, self.N))
    
    def transform(self, corpus: list) -> Tuple[np.ndarray, np.ndarray]:
        X_ngrams = []
        y = []
        for doc in corpus:
            doc_ngram = self.get_ngram_doc(doc)
            for words_window in doc_ngram:
                words_window_ids = [self.w2id[w] for w in words_window]
                X_ngrams.append(list(words_window_ids[:-1]))
                y.append(words_window_ids[-1])
        return np.array(X_ngrams), np.array(y)

In [46]:
tk = TweetTokenizer()
ngram_data = NgramData(args.N, 5000, tk.tokenize)
ngram_data.fit(X_train)

In [96]:
tk = TweetTokenizer()
ngram_data = NgramData(args.N, 5000, tk.tokenize, char_level=True)
ngram_data.fit(X_train)

In [97]:
print('Vocab Size:', ngram_data.get_vocab_size())

Vocab Size: 344


In [98]:
ngram_data.vocab

{' ',
 '#',
 '$',
 '%',
 '&',
 '(',
 ')',
 '+',
 '/',
 '<',
 '</s>',
 '<s>',
 '<unk>',
 '=',
 '>',
 '@',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '\xa0',
 '¬',
 '°',
 '´',
 'à',
 'á',
 'é',
 'í',
 'ñ',
 'ó',
 'ú',
 'ü',
 'ʺ',
 'ˮ',
 '̶',
 '\u200d',
 '–',
 '—',
 '‘',
 '’',
 '“',
 '”',
 '‟',
 '•',
 '…',
 '″',
 '‼',
 '‿',
 '↪',
 '►',
 '◕',
 '☀',
 '☔',
 '☕',
 '☝',
 '☹',
 '☺',
 '♀',
 '♂',
 '♡',
 '♥',
 '⚡',
 '⚽',
 '✊',
 '✋',
 '✌',
 '✍',
 '✔',
 '✨',
 '❌',
 '❕',
 '❣',
 '❤',
 '⭐',
 'づ',
 '・',
 '과',
 '기',
 '력',
 '사',
 '소',
 '스',
 '엑',
 '워',
 '원',
 '자',
 '전',
 '츠',
 '파',
 '포',
 '해',
 '️',
 '🇦',
 '🇨',
 '🇪',
 '🇭',
 '🇮',
 '🇱',
 '🇲',
 '🇳',
 '🇷',
 '🇸',
 '🇺',
 '🇽',
 '🌈',
 '🌑',
 '🌚',
 '🌟',
 '🌩',
 '🌶',
 '🌹',
 '🌺',
 '🌼',
 '🌾',
 '🍆',
 '🍌',
 '🍑',
 '🍳',
 '🍴',
 '🍷',
 '🍸',
 '🍻',
 '🎂',
 '🎃',
 '🎈',
 '🎉',
 '🎒',
 '🎤',
 '🎭',
 '🎵',
 '🎶',
 '🎼',
 '🏈',
 '🏠',
 '🏨',
 '🏻',
 '🏼',

In [99]:
X_ngram_train, y_ngram_train = ngram_data.transform(X_train)
X_ngram_val, y_ngram_val = ngram_data.transform(X_val)

In [100]:
#Batch size
args.batch_size = 64
#Number workers
args.num_workers = 2

#Train
train_dataset = TensorDataset(torch.tensor(X_ngram_train, dtype=torch.int64),
                 torch.tensor(y_ngram_train, dtype=torch.int64))
train_loader = DataLoader(train_dataset,
                          batch_size = args.batch_size,
                          num_workers = args.num_workers,
                          shuffle = True)

#Validation
val_dataset = TensorDataset(torch.tensor(X_ngram_val, dtype=torch.int64),
                 torch.tensor(y_ngram_val, dtype=torch.int64))
val_loader = DataLoader(val_dataset,
                          batch_size = args.batch_size,
                          num_workers = args.num_workers,
                          shuffle = False)

In [101]:
batch = next(iter(train_loader))
print('X shape:', batch[0].shape)
print('y shape:', batch[1].shape)

X shape: torch.Size([64, 5])
y shape: torch.Size([64])


In [102]:
[[ngram_data.id2w[w] for w in tw] for tw in batch[0].tolist()]

[['c', 'u', 'a', 'n', 'd'],
 ['i', 'l', 'i', 'b', 'r'],
 ['j', 'a', 'j', 'a', 'q'],
 ['i', 'm', 'a', 'd', 'r'],
 ['i', 'o', 'c', 'l', 'a'],
 ['o', 's', '<', 'u', 'n'],
 ['i', 'a', 'n', 'd', 'o'],
 ['r', 't', 'ó', 'e', 'l'],
 ['a', '<', 'u', 'n', 'k'],
 ['e', 'l', 'a', 'n', 'o'],
 ['z', 'q', 'u', 'e', 't'],
 ['u', 'n', 'k', '>', 'f'],
 ['s', 'o', 's', 'e', 's'],
 ['o', 'n', 's', 'u', 'c'],
 ['a', 'e', 'n', 'c', 'u'],
 ['u', 'n', 'd', 'í', 'a'],
 ['o', 'c', 'o', '<', 'u'],
 ['>', '<', 'u', 'n', 'k'],
 ['a', 'n', 'o', 'q', 'u'],
 ['u', 'm', 'a', 'd', 'r'],
 ['n', 'v', 'e', 'n', 'c'],
 ['d', 'y', 'q', 'u', 'e'],
 ['i', 't', 'a', '😂', '🙊'],
 ['c', 'a', 'b', 'a', 'r'],
 ['p', 'i', 'c', 'ó', 'h'],
 ['c', 'i', 'a', 's', 'j'],
 ['c', 'e', 'r', 'a', 'm'],
 ['i', 'p', 'r', 'o', 'p'],
 ['e', 'g', 'a', 'n', 'a'],
 ['b', 'o', 'c', 'a', 'd'],
 ['r', 'a', 'm', 'e', 'j'],
 ['<s>', '<s>', '<s>', '<s>', '<s>'],
 ['<s>', '<s>', 'd', 'e', 'b'],
 ['m', 'o', '<', 'u', 'n'],
 ['a', 't', 'o', 'd', 'o'],
 ['<s>

In [103]:
#Vocabulary size
args.vocab_size = ngram_data.get_vocab_size()
#Word embeddings dimension
args.d = 50
#Hidden layer dimension
args.d_h = 100
#Dropout
args.dropout = 0.1

In [104]:
class NeuralLM(nn.Module):
    def __init__(self, args):
        super(NeuralLM, self).__init__()

        self.window_size = args.N - 1
        self.embedding_size = args.d

        self.emb = nn.Embedding(args.vocab_size, args.d)
        self.fc1 = nn.Linear(args.d * (args.N - 1), args.d_h)
        self.drop1 = nn.Dropout(p = args.dropout)
        self.fc2 = nn.Linear(args.d_h, args.vocab_size, bias=False)

    def forward(self, x):
        x = self.emb(x)
        x = x.view(-1, self.window_size * self.embedding_size)
        h = F.relu(self.fc1(x))
        h = self.drop1(h)
        x = self.fc2(h)
        return x

In [105]:
def get_preds(raw_logits):
    probs = F.softmax(raw_logits.detach(), dim=1)
    y_pred = torch.argmax(probs, dim=1).cpu().numpy()
    return y_pred

def model_eval(data, model, gpu=False):
    with torch.no_grad():
        preds, tgts = [], []
        for window_words, labels in data:
            if gpu:
                window_words = window_words.cuda()
            outputs = model(window_words)

            #Predictions
            y_pred = get_preds(outputs)
            tgt = labels.numpy()
            tgts.append(tgt)
            preds.append(y_pred)
    tgts = [e for l in tgts for e in l]
    preds = [e for l in preds for e in l]
    return accuracy_score(tgts, preds)

def save_checkpoint(state, is_best, checkpoint_path, filename='checkpoint.pt'):
    filename = os.path.join(checkpoint_path, filename)
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, os.path.join(checkpoint_path, 'model_best.pt'))

In [106]:
#Model hyperparameters
args.vocab_size = ngram_data.get_vocab_size()
args.d = 100
args.d_h = 200
args.dropout = 0.1

#Training hyperparameters
args.lr = 2.3e-1
args.num_epochs = 100
args.patience = 20

#Scheduler hyperparameters
args.lr_patience = 10
args.lr_factor = 0.5

#Saving hyperparameters
args.savedir = pth + 'model'
os.makedirs(args.savedir, exist_ok=True)

#Create model
model = NeuralLM(args=args)

#Send to GPU
args.use_gpu = torch.cuda.is_available()
if args.use_gpu:
    model.cuda()

#Loss, Optimizer and Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer = optimizer, 
                                                       mode = 'min',
                                                       factor = args.lr_factor,
                                                       patience = args.lr_patience,
                                                       verbose = True)

In [107]:
start_time = time.time()
best_metric = 0
metric_history = []
train_metric_history = []

#Training
for epoch in range(args.num_epochs):
    epoch_start_time = time.time()
    loss_epoch = []
    training_metric = []
    model.train()
    for window_words, labels in train_loader:
        if args.use_gpu:
            window_words = window_words.cuda()
            labels = labels.cuda()

        #Forward pass
        outputs = model(window_words)
        loss = criterion(outputs, labels)
        loss_epoch.append(loss.item())

        #Get training metrics
        y_pred = get_preds(outputs)
        tgt = labels.cpu().numpy()
        training_metric.append(accuracy_score(tgt, y_pred))

        #Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    #Metrics in training dataset
    mean_epoch_metric = np.mean(training_metric)
    train_metric_history.append(mean_epoch_metric)

    #Metrics in validation dataset
    model.eval()
    tuning_metric = model_eval(val_loader, model, gpu=args.use_gpu)
    mean_epoch_metric = np.mean(tuning_metric)
    metric_history.append(mean_epoch_metric)

    #Update scheduler
    scheduler.step(tuning_metric)

    #Check metric improvement
    is_improve = tuning_metric > best_metric
    if is_improve:
        best_metric = tuning_metric
        n_no_improve = 0
    else:
        n_no_improve += 1

    #Save best model
    save_checkpoint({'epoch'       : epoch+1,
                     'state_dict'  : model.state_dict(),
                     'optimizer'   : optimizer.state_dict(),
                     'scheduler'   : scheduler.state_dict(),
                     'best_metric' : best_metric}, is_improve, args.savedir)
    
    #Early stoping
    if n_no_improve >= args.patience:
        print('No improvement. Breaking out of loop.')
        break
    print('Train accuracy: ', mean_epoch_metric)
    print('Epoch [{}/{}]: Loss = {:.4f}, Val Acurracy = {:.4f}, Epoch time = {:.2f}'.
          format(epoch+1, args.num_epochs, np.mean(loss_epoch), tuning_metric, (time.time()-epoch_start_time)))
    
print('---------- %s seconds ---------' % time.time()-start_time)

Train accuracy:  0.40961413917730377
Epoch [1/100]: Loss = 2.1350, Val Acurracy = 0.4096, Epoch time = 27.72
Train accuracy:  0.421041636200305
Epoch [2/100]: Loss = 1.9964, Val Acurracy = 0.4210, Epoch time = 26.87
Train accuracy:  0.43132011615517996
Epoch [3/100]: Loss = 1.9522, Val Acurracy = 0.4313, Epoch time = 26.83
Train accuracy:  0.4414314663546911
Epoch [4/100]: Loss = 1.9236, Val Acurracy = 0.4414, Epoch time = 26.83
Train accuracy:  0.45129212192115653
Epoch [5/100]: Loss = 1.9025, Val Acurracy = 0.4513, Epoch time = 26.72
Train accuracy:  0.4493910209538931
Epoch [6/100]: Loss = 1.8875, Val Acurracy = 0.4494, Epoch time = 26.78
Train accuracy:  0.4501639960724507
Epoch [7/100]: Loss = 1.8747, Val Acurracy = 0.4502, Epoch time = 26.77
Train accuracy:  0.4533603526437838
Epoch [8/100]: Loss = 1.8651, Val Acurracy = 0.4534, Epoch time = 26.84
Train accuracy:  0.45432134873712576
Epoch [9/100]: Loss = 1.8564, Val Acurracy = 0.4543, Epoch time = 26.62
Train accuracy:  0.455428

KeyboardInterrupt: 

In [108]:
def print_closest_words(embeddings, ngram_data, word, n):
    word_id = torch.LongTensor([ngram_data.w2id[word]])
    word_embed = embeddings(word_id)
    dists = torch.norm(embeddings.weight - word_embed, dim=1).detach()
    lst = sorted(enumerate(dists.numpy()), key=lambda x : x[1])
    for idx, diff in lst[1:n+1]:
        print(ngram_data.id2w[idx], diff)

In [110]:
#Model with learned embeddings
best_model = NeuralLM(args)
best_model.load_state_dict(torch.load(pth+'model/model_best.pt')['state_dict'])
best_model.train(False)

print('Learned embeddings')
print('¯'*20)
print_closest_words(best_model.emb, ngram_data, ' ', 10)

Learned embeddings
¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯
e 9.830577
i 10.166203
o 10.178575
n 10.231117
u 10.306223
a 10.354281
<s> 10.3785715
r 10.484237
t 10.558488
y 10.7990675


In [118]:
def parse_text(text, tokenizer):
    all_tokens = [w.lower() if w in ngram_data.w2id else '<unk>' for w in tokenizer.tokenize(text)]
    token_ids = [ngram_data.w2id[word.lower()] for word in all_tokens]
    return all_tokens, token_ids

def sample_next_word(logits, temperature=1.):
    logits = np.asarray(logits).astype('float64')
    preds = logits/temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probs = np.random.multinomial(1, preds)
    return np.argmax(probs)

def predict_next_token(model, token_ids):
    words_ids_tensor = torch.LongTensor(token_ids).unsqueeze(0)
    y_raw_pred = model(words_ids_tensor).squeeze(0).detach().numpy()
    y_pred = sample_next_word(y_raw_pred, 1.)
    return y_pred

def generate_sentence(model, initial_text, tokenizer, char=False):
    all_tokens, window_word_ids = parse_text(initial_text, tokenizer)
    for i in range(500):
        y_pred = predict_next_token(best_model, window_word_ids)
        next_word = ngram_data.id2w[y_pred]
        all_tokens.append(next_word)
        if next_word == '</s>':
            break
        else:
            window_word_ids.pop(0)
            window_word_ids.append(y_pred)
    if char:
        return ' '.join(all_tokens)
    else:
        return ' '.join(all_tokens)

In [119]:
initial_tokens = '<s><s><s><s><s>'
print('Learned embeddings')
print('¯'*20)
generate_sentence(best_model, initial_tokens, tk, char=True)

Learned embeddings
¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯


'<s> <s> <s> <s> <s> a m i g o l < u n k > < u n k > < u n k > m i g u t a x y p i d e a </s>'

In [25]:
initial_tokens = '<s> <s> estoy'
print('Learned embeddings')
print('¯'*20)
generate_sentence(best_model, initial_tokens, tk)

Learned embeddings
¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯


'<s> <s> estoy quitar <unk> ” <unk> que pena harto #putita regresa de <unk> a ver si se me ocurre la <unk> 😂 😂 😂 hijos de gustar <unk> seguro <unk> <unk> <unk> de <unk> <unk> <unk> saludos gordo se <unk> <unk> <unk> cambia <unk> <unk> de copa que bailar frases tiene <unk> mi <unk> <unk> es exige te digan se vale <unk> hdp se <unk> <unk> </s>'

In [26]:
initial_tokens = 'yo opino que'
print('Learned embeddings')
print('¯'*20)
generate_sentence(best_model, initial_tokens, tk)

Learned embeddings
¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯


'yo opino que de lo que <unk> 😂 😂 😂 </s>'

In [28]:
def log_likelihood(model, text, ngram_model):
    #n-gram windows 
    X, y = ngram_data.transform([text])
    #Discard first two n-gram windows since they contain '<s>' tokens not necessary
    X, y = X[2:], y[2:]
    X = torch.LongTensor(X).unsqueeze(0)

    logits = model(X).detach()
    probs = F.softmax(logits, dim = 1).numpy()

    return np.sum([np.log(probs[i][w]) for i, w in enumerate(y)])

In [29]:
print("Log likelihood: ", log_likelihood(best_model, "Estamos en la clase de procesamiento de lenguaje", ngram_data))

Log likelihood:  -18.708864


In [30]:
print("Log likelihood: ", log_likelihood(best_model, "Estamos procesamiento clase en la de natural de lenguaje", ngram_data))

Log likelihood:  -32.64714


In [31]:
print("Log likelihood: ", log_likelihood(best_model, "la natural Estamos clase en de de lenguaje procesamiento", ngram_data))

Log likelihood:  -48.377045


# Estructuras sintácticas correctas

In [32]:
word_list = "sino gano me voy a la chingada".split(' ')
perms = [' '.join(perm) for perm in permutations(word_list)]
#print(len(perms))

for p, t in sorted([(log_likelihood(best_model, text, ngram_data), text) for text in perms], reverse=True)[:5]:
    print(p, t)

print('-'*50)
for p, t in sorted([(log_likelihood(best_model, text, ngram_data), text) for text in perms], reverse=True)[-5:]:
    print(p, t)

-21.248997 gano sino me voy a la chingada
-22.199257 sino gano me voy a la chingada
-25.700716 sino voy a me la chingada gano
-25.700972 sino gano voy a me la chingada
-25.718216 voy gano me sino a la chingada
--------------------------------------------------
-71.674545 me a sino gano voy chingada la
-71.88536 me a sino voy gano chingada la
-72.65167 me a voy sino gano chingada la
-73.53817 voy me a sino gano chingada la
-77.574425 me a sino gano chingada la voy
